In [ ]:
import numpy as np
from typing import List, Tuple, Dict, Any

#  Интервальная реализация VIKOR
def interval_vikor(
    matrix: np.ndarray,          
    weights: List[float],        
    types: List[int],            
    v: float = 0.5
) -> Dict[str, Any]:
    # Возвращает интервальные S, R, Q и ранжирование.

    m, n = matrix.shape[0], matrix.shape[1]
    L = matrix[:, :, 0]
    U = matrix[:, :, 1]

    # 1) интервальные идеал/надир для каждого критерия
    f_star = np.zeros((n, 2))   # [low, high]
    f_minus = np.zeros((n, 2))  # [low, high]

    for j in range(n):
        if types[j] == 1:  # benefit
            f_star[j, 0] = np.max(L[:, j])   # best lower
            f_star[j, 1] = np.max(U[:, j])   # best upper
            f_minus[j, 0] = np.min(L[:, j])  # worst lower
            f_minus[j, 1] = np.min(U[:, j])  # worst upper
        else:  # cost
            f_star[j, 0] = np.min(L[:, j])
            f_star[j, 1] = np.min(U[:, j])
            f_minus[j, 0] = np.max(L[:, j])
            f_minus[j, 1] = np.max(U[:, j])

    # 2) вычисляем интервальные нормализованные разрывы D_ij = [D_low, D_high]
    # D_low = (f*_low - overline f_ij) / (f*_low - f-_high)
    # D_high = (f*_high - underline f_ij) / (f*_high - f-_low)
    # будет применяться для каждого критерия и альтернативы.
    D_low = np.zeros((m, n))
    D_high = np.zeros((m, n))

    for i in range(m):
        for j in range(n):
            fstar_low = f_star[j, 0]
            fstar_high = f_star[j, 1]
            fminus_low = f_minus[j, 0]
            fminus_high = f_minus[j, 1]

            over_ij = U[i, j]
            under_ij = L[i, j]

            denom_low = (fstar_low - fminus_high)
            denom_high = (fstar_high - fminus_low)

            if denom_low == 0:
                d_low = 0.0
            else:
                d_low = (fstar_low - over_ij) / denom_low

            if denom_high == 0:
                d_high = 0.0
            else:
                d_high = (fstar_high - under_ij) / denom_high

            D_low[i, j] = np.clip(d_low, 0.0, 1.0)
            D_high[i, j] = np.clip(d_high, 0.0, 1.0)

    # 3) агрегируем по весам: интервальные S_i и R_i
    weights = np.array(weights)
    S = np.zeros((m, 2))  # [S_low, S_high]
    R = np.zeros((m, 2))  # [R_low, R_high]

    for i in range(m):
        # суммирование по низким и по высоким границам
        S[i, 0] = np.sum(weights * D_low[i, :])
        S[i, 1] = np.sum(weights * D_high[i, :])
        # R = max_j w_j * D_ij (по компонентам)
        R[i, 0] = np.max(weights * D_low[i, :]) if n > 0 else 0.0
        R[i, 1] = np.max(weights * D_high[i, :]) if n > 0 else 0.0

    # 4) вычисляем S*, S-, R*, R- для комбинаций границ
    S_star_low = np.min(S[:, 0])
    S_star_high = np.min(S[:, 1])
    S_minus_low = np.max(S[:, 0])
    S_minus_high = np.max(S[:, 1])

    R_star_low = np.min(R[:, 0])
    R_star_high = np.min(R[:, 1])
    R_minus_low = np.max(R[:, 0])
    R_minus_high = np.max(R[:, 1])

    # 5) вычисляем интервальный Q: левый и правый концы
    Q = np.zeros((m, 2))
    # предохраняемся от деления на ноль
    def safe_div(numer, denom):
        return numer / denom if abs(denom) > 1e-12 else 0.0

    for i in range(m):
        # левый (пессимистичный)
        num_S_left = S[i, 0] - S_star_high
        den_S_left = S_minus_low - S_star_high
        term_S_left = safe_div(num_S_left, den_S_left)

        num_R_left = R[i, 0] - R_star_high
        den_R_left = R_minus_low - R_star_high
        term_R_left = safe_div(num_R_left, den_R_left)

        q_left = v * term_S_left + (1.0 - v) * term_R_left

        # правый (оптимистичный)
        num_S_right = S[i, 1] - S_star_low
        den_S_right = S_minus_high - S_star_low
        term_S_right = safe_div(num_S_right, den_S_right)

        num_R_right = R[i, 1] - R_star_low
        den_R_right = R_minus_high - R_star_low
        term_R_right = safe_div(num_R_right, den_R_right)

        q_right = v * term_S_right + (1.0 - v) * term_R_right

        # защита: Q в [0,1]
        Q[i, 0] = np.clip(q_left, 0.0, 1.0)
        Q[i, 1] = np.clip(q_right, 0.0, 1.0)

    # 6) ранжирование (по центрам интервалов Q)
    Q_centers = (Q[:, 0] + Q[:, 1]) / 2.0
    ranking = np.argsort(Q_centers)

    # 7) условия приемлемости на центрах
    sorted_centers = Q_centers[ranking]
    DQ = 1.0 / (m - 1) if m > 1 else 1.0
    condition1 = (len(sorted_centers) > 1 and (sorted_centers[1] - sorted_centers[0]) >= DQ)

    S_centers = (S[:, 0] + S[:, 1]) / 2.0
    R_centers = (R[:, 0] + R[:, 1]) / 2.0
    best_idx = int(ranking[0])
    condition2 = (best_idx == int(np.argmin(S_centers)) or best_idx == int(np.argmin(R_centers)))

    best_alternative = best_idx
    if not (condition1 and condition2) and m > 1:
        best_alternative = [int(ranking[0]), int(ranking[1])]

    return {
        'ranking': ranking.tolist(),
        'Q': Q.tolist(),
        'S': S.tolist(),
        'R': R.tolist(),
        'best_alternative': best_alternative,
        'conditions_met': bool(condition1 and condition2),
        'Q_centers': Q_centers.tolist(),
        'f_star': f_star.tolist(),
        'f_minus': f_minus.tolist(),
        'D_low': D_low.tolist(),
        'D_high': D_high.tolist()
    }


if __name__ == "__main__":
    matrix = np.array([
        [[0.75, 1.24], [2784, 3192]],  # A1
        [[1.83, 2.11], [3671, 3857]],  # A2
        [[4.90, 5.73], [4409, 4681]],  # A3
    ])
    weights = [0.5, 0.5]
    types = [-1, 1]
    res = interval_vikor(matrix, weights, types, v=0.5)
    import pprint
    pprint.pprint(res)


{'D_high': [[0.0, 1.0],
            [0.16120218579234974, 0.5324196099103848],
            [1.0, 0.143384290985767]],
 'D_low': [[0.09839357429718874, 1.0],
           [0.2730923694779116, 0.45357436318816763],
           [1.0, 0.0]],
 'Q': [[1.0, 0.8279598637936816],
       [0.0, 0.03249728323936535],
       [0.8784579613741175, 1.0]],
 'Q_centers': [0.9139799318968408, 0.016248641619682674, 0.9392289806870587],
 'R': [[0.5, 0.5], [0.22678718159408381, 0.2662098049551924], [0.5, 0.5]],
 'S': [[0.5491967871485943, 0.5],
       [0.3633333663330396, 0.3468108978513673],
       [0.5, 0.5716921454928835]],
 'best_alternative': 1,
 'conditions_met': True,
 'f_minus': [[4.9, 5.73], [2784.0, 3192.0]],
 'f_star': [[0.75, 1.24], [4409.0, 4681.0]],
 'ranking': [1, 0, 2]}
